In [1]:
# I didn't wind up using this in my submissions, but I think this is a better CV splitter, I just ran out of time because of XGB issues.

import pickle
import pandas as pd
import numpy as np
import gzip
import datetime
from itertools import product
from scipy import interpolate ## For other interpolation functions.
import time

import sklearn.metrics

from sklearn.cross_validation import LabelKFold

import copy


/opt/conda/lib/python3.5/site-packages/sklearn/cross_validation.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
with gzip.open('merged7.pkl.gz', 'rb') as fd:
    data = pickle.load(fd)

In [3]:
testset = np.where(data['outcome'].isnull())
test = data.iloc[testset]

trainset = np.where(~data['outcome'].isnull())
train = data.iloc[trainset]

In [4]:
# compute the 'second' leak

# ALL test targets with 0.0/1.0 average are fully/accurately inferred in the test set

grouplist_all0 = []
grouplist_all1 = []
grouplist = []
count = 0

for g in data.groupby(['group_1'], sort=False):
    if len(g[1]) > 50:
        m = g[1].outcome.mean()
        if m != 0 and m != 1:
            #print(g[0], len(g[1]), g[1].outcome.mean())
            grouplist.append(g[0])
            
            count += len(g[1])
        elif m == 0:
            grouplist_all0.append(g[0])
        elif m == 1:
            grouplist_all1.append(g[0])
    else:
        grouplist.append(g[0])
            
        count += len(g[1])
            
print(len(grouplist), len(grouplist_all0), len(grouplist_all1), count)

29281 1393 3550 756155


In [5]:
mask0 = np.full(len(data), False, dtype=np.bool)
mask1 = np.full(len(data), False, dtype=np.bool)
for g in grouplist_all0:
    mask0 = np.logical_or(mask0, data.group_1 == g)
    #train.outcome_filled.values[np.where(mask)] = .05
for g in grouplist_all1:
    mask1 = np.logical_or(mask1, data.group_1 == g)
    #train.outcome_filled.values[np.where(mask)] = .95

In [6]:
data['outcome_leak2'] = np.full_like(data['outcome'].values, np.nan)

data.loc[mask0, 'outcome_leak2'] = .05
data.loc[mask1, 'outcome_leak2'] = .95

testset = np.where(data['outcome'].isnull())
test = data.iloc[testset]

trainset = np.where(~data['outcome'].isnull())
train = data.iloc[trainset]

In [7]:
leak2_mask = np.logical_or(mask0, mask1)

In [8]:
cols = train.columns.copy()
cols = cols.drop('activity_id')
train_dups = train.duplicated(subset=cols)
train_dedup = train[~train_dups]

In [9]:
print(len(train_dedup), len(train))

1213878 2197291


In [10]:
from operator import itemgetter

stratkey = 'people_id'

balls = []
for p in train_dedup.groupby([stratkey]):
    balls.append([p[0], len(p[1]), p[1].outcome.mean()])

balls = sorted(balls, key=itemgetter(2), reverse=True)

# This assumes vc is sorted by whatever you want stratified
def dosplit_rr(df, vc, folds, fuzz = (43254, .5, 4)):
    if fuzz is not None:
        np.random.seed(fuzz[0])
    
    bcount = np.zeros(folds)
    
    buckets = []
    for f in range(folds):
        buckets.append([])
    
    ballpit = copy.deepcopy(balls)
    
    runs = 0
    
    tot = 0
    
    while len(ballpit):
        runs += 1
        sel = 0
        r = np.random.rand()
        if r < fuzz[1]:
            sel = int((fuzz[2] / fuzz[1]) * r)
            if sel >= len(ballpit):
                sel = len(ballpit) - 1

        tot += sel
                
        v = ballpit[sel]
        del ballpit[sel]
                
        selbucket = np.argsort(bcount)[0]
        
        buckets[selbucket].append(v[0])
        bcount[selbucket] += v[1]
    
    print(len(balls), runs, tot)
    
    return buckets
    

In [12]:
folds = 8
pidsets_grouped = dosplit_rr(train_dedup, balls, folds)

151295 151295 113607


In [13]:
pidset = pidsets_grouped

In [14]:
cv_train = []
cv_val = []
cv_val_tgt = []

cv_val_dups, cv_val_dups_tgt = [], []

pu = []

for p in pidset:
    cv_train.append(train_dedup[~train_dedup.people_id.isin(p)])

    cv_val.append(train_dedup[train_dedup.people_id.isin(p)])
    cv_val_tgt.append(cv_val[-1][['activity_id', 'outcome']].copy())
    
    pu.append(list(cv_val[-1].people_id.unique()))

    cv_val_dups.append(train[train.people_id.isin(p)])
    cv_val_dups_tgt.append(cv_val_dups[-1][['activity_id', 'outcome']].copy())


In [15]:
for k in range(len(cv_val)):
    print(k, len(cv_val_tgt[k]), cv_val_tgt[k].outcome.mean(), len(cv_val_dups_tgt[k]), cv_val_dups_tgt[k].outcome.mean())

0 151730 0.48606076583404734 277430 0.43994881591752877
1 151737 0.4860383426586792 256566 0.50167208437595
2 151740 0.48605509424014764 293694 0.4056535033061622
3 151727 0.4860703763997179 306867 0.3978205541814532
4 151733 0.48605115564840873 252029 0.476675303238913
5 151739 0.4860319364171373 252860 0.477449972316697
6 151742 0.48604209777121693 254765 0.4735933114831315
7 151730 0.4860805377974033 303080 0.40251748713211033


In [16]:
# Process dates

if True: # Wasteful to recompute a constant every time
    minactivdate = pd.Timestamp('2022-07-17 00:00:00')
    maxactivdate = pd.Timestamp('2023-08-31 00:00:00')
else:
    minactivdate = min(activs['date'])
    maxactivdate = max(activs['date'])

alldays = [maxactivdate - datetime.timedelta(days=x) for x in range(0, (maxactivdate - minactivdate).days+1)][::-1]

In [17]:
fv = []

def interpolateFun0(x):
    """Original script author's function rewritten in Python.
    The author interpolates between two known values by averaging them. We
    can think of this as 0th order interpolation. """

    ## TODO: This function could use some optimization. The R version is much faster...
    x = x.reset_index(drop=True)
    g = x['outcome'].copy() ## g should be a list or a pandas Series.
    
    global fv

    if (g.shape[0] < 3): ## If we have at most two rows.
        x['outcome_filled'] = g ## Will be replaced by a mean.
#        x['outcome'] = x['filled']
        return x
    
    if np.sum(g.isnull()) == 0:
        x['outcome_filled'] = g
        return x
    
    out = g.values.copy()
    value_locs = np.where(~g.isnull())[0]
    
    if len(value_locs) == 0:
        x['outcome_filled'] = np.full_like(out, np.nan)
#        x['outcome'] = x['filled']
        return x
    
    if len(value_locs) == 1:
        fillval = .89 if (g[value_locs[0]] == 1) else .13
        fv.append((g[value_locs[0]], fillval))
        g[g.isnull()] = fillval

        x['outcome_filled'] = g
#        x['outcome'] = x['filled']

        return x        
    
    # Fill in beginning (if needed)
    if value_locs[0]:
        
        fillval = .89 if (g[value_locs[0]] == 1) else .13
        fv.append((g[value_locs[0]], fillval))
        
        out[0:value_locs[0]] = fillval

    # Interpolate holes in the middle
    for i in range(0, len(value_locs) - 1):
        beg = value_locs[i]
        end = value_locs[i + 1]
        
        if g[beg] != g[end]:
            out[beg+1:end] = np.interp(range(beg+1, end), [beg, end], [g[beg], g[end]])
        else:
            out[beg+1:end] = g[beg]

    # Fill in end (if needed)
    if end < (len(g) - 1):
        beg = value_locs[-1]
        fillval = .89 if (g[beg] == 1) else .13
        fv.append((g[beg], fillval))

        out[beg+1:] = fillval

    x['outcome_filled'] = out
#    x['outcome'] = x['filled']
    
    return x


In [18]:
cv_mean = []
leak = []

for k in range(len(cv_val)):
#for k in [0]:
    cv_mean.append(cv_train[k].outcome.mean())
    
    groups = cv_val[k].group_1.unique()

    allGroupsAndDays = pd.DataFrame.from_records(product(groups, alldays))
    allGroupsAndDays.columns = ['group_1', 'adate_leak']

    meanbycomdate = cv_train[k].groupby(['group_1', 'adate'])['outcome'].agg('mean')

    ## Convert the calculation into a proper DataFrame.
    meanbycomdate = meanbycomdate.to_frame().reset_index()
    meanbycomdate.rename(columns={'adate': 'adate_mean'}, inplace=True)

    allGroupsAndDays = pd.merge(allGroupsAndDays, meanbycomdate, left_on=['group_1', 'adate_leak'], right_on=['group_1', 'adate_mean'], how='left')

    agad2 = allGroupsAndDays.groupby('group_1').apply(interpolateFun0)
    agad2 = agad2.rename(columns={'outcome': 'outcome_leak'})
    agad2 = agad2.drop(['adate_mean'], axis=1)
    
    cv_val_dups[k] = pd.merge(cv_val_dups[k], agad2, left_on=['group_1', 'adate'], right_on=['group_1', 'adate_leak'], how='left')
    cv_val_dups[k].drop('adate_leak', axis=1, inplace=True)
    

In [19]:
t = cv_train[0]
t.outcome.mean()

ta = t[t.outcome_leak2.isnull()]
len(ta)
t.outcome.mean(), ta.outcome.mean()

(0.4860527911364518, 0.5974661947663344)

In [20]:
len(cv_val_dups[3]), len(cv_val[3])

(306867, 151727)

In [21]:
cv_val_dups_tgt_wl = []

for k in range(len(cv_val_dups)):
        
    f = pd.merge(cv_val_dups_tgt[k], cv_val_dups[k][['activity_id', 'outcome_filled', 'outcome_leak', 'outcome_leak2']], on=['activity_id'], how='left')
    
    tr = cv_train[k]
    trl2 = tr[tr.outcome_leak2.isnull()]
    
    print(tr.outcome.mean(), trl2.outcome.mean())
    
    #mask = np.where(~f.outcome_leak2.isnull())
    #f.outcome_filled.values[mask] = f.outcome_leak2.values[mask]
    
    f['outcome_filled_nona'] = f.outcome_filled.fillna(tr.outcome.mean())

    cv_val_dups_tgt_wl.append(f)
    
    print(sklearn.metrics.roc_auc_score(f.outcome.values, f.outcome_filled_nona.values))

#tmp0 = cv_val_tgt_wl[[0, 2, 3, 4, 5]]
tmp = pd.concat(cv_val_dups_tgt_wl)
print('cv:', sklearn.metrics.roc_auc_score(tmp.outcome.values, tmp.outcome_filled_nona.values))

0.4860527911364518 0.5974661947663344
0.987178087269
0.4860559944489479 0.5991144314513376
0.987662726824
0.4860536013211089 0.5996463392871627
0.990469338006
0.48605141830116433 0.5971859042156283
0.990029341263
0.4860541639794943 0.5984634204752134
0.987749075407
0.48605690968884485 0.5992141008322238
0.987918958579
0.4860554580581018 0.5988555310595254
0.984852696122
0.48604996667131134 0.5991608318935842
0.969472140622
cv: 0.987745455254


In [22]:
train_withleak = pd.merge(train, tmp[['activity_id', 'outcome_filled', 'outcome_filled_nona', 'outcome_leak']], on='activity_id', how='left')

print('cv:', sklearn.metrics.roc_auc_score(train_withleak.outcome.values, train_withleak.outcome_filled_nona.values))

cv: 0.987745455254


In [23]:
train_leak = train_withleak[['activity_id', 'outcome_filled', 'outcome_leak']]

In [24]:
len(train_leak)

2197291

In [25]:
train_leak.to_pickle('train_cvleak8.pkl')

In [26]:
# produce test leak

In [27]:
groups = train.group_1.unique()

allGroupsAndDays = pd.DataFrame.from_records(product(groups, alldays))
allGroupsAndDays.columns = ['group_1', 'adate_leak']

meanbycomdate = train.groupby(['group_1', 'adate'])['outcome'].agg('mean')

## Convert the calculation into a proper DataFrame.
meanbycomdate = meanbycomdate.to_frame().reset_index()
meanbycomdate.rename(columns={'adate': 'adate_mean'}, inplace=True)

allGroupsAndDays = pd.merge(allGroupsAndDays, meanbycomdate, left_on=['group_1', 'adate_leak'], right_on=['group_1', 'adate_mean'], how='left')

agad2 = allGroupsAndDays.groupby('group_1').apply(interpolateFun0)
agad2 = agad2.rename(columns={'outcome': 'outcome_leak'})
agad2 = agad2.drop(['adate_mean'], axis=1)

In [28]:
test_withleak = pd.merge(test, agad2, left_on=['group_1', 'adate'], right_on=['group_1', 'adate_leak'], how='left')
test_withleak.drop('adate_leak', axis=1, inplace=True)

In [29]:
train_mean = train.outcome.mean()
train_l2 = train[train.outcome_leak2.isnull()]
#tr = cv_train[k]
#trl2 = tr[tr.outcome_leak2.isnull()]
#train_mean  = train_l2.outcome.mean()

In [30]:
#mask = np.where(~test.outcome_leak2.isnull())
#test_withleak.outcome_filled.values[mask] = test_withleak.outcome_leak2.values[mask]

In [31]:
test_withleak.outcome_filled.isnull().sum()

69073

In [32]:
test_withleak['outcome_filled_nona'] = test_withleak.outcome_filled.fillna(train_mean)

In [33]:
test_out =  test_withleak[['activity_id', 'outcome_filled_nona']].copy()

In [34]:
test_out.rename(columns={'outcome_filled_nona':'outcome'}, inplace=True)

In [35]:
test_out.to_csv('Submission-cvleak8.csv', index=False)

In [36]:
train_leak['outcome_filled_nona'] = train_withleak.outcome_filled_nona

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [37]:
# Now merge the leak

test_leak = test_withleak[['activity_id', 'outcome_filled', 'outcome_filled_nona', 'outcome_leak']]

all_leak = pd.concat([train_leak, test_leak])

all_leak['outcome_filled_nona'][:len(train_leak)] = train_withleak['outcome_filled_nona']
#all_leak.iloc[0:len(train_leak), ['outcome_filled_nona']] = train_withleak['outcome_filled_nona'].values.copy()
all_leak['outcome_leak_int'] = all_leak['outcome_leak'].copy()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [38]:
# ... and copy 0.0/1.0 from outcome_filled to a new entry with likely-good middles filled in
mask = all_leak['outcome_filled'] == 1.0
mask = np.logical_or(mask, all_leak['outcome_filled'] == 0.0)

#np.sum(mask)

all_leak.outcome_leak_int.values[np.where(mask)] = all_leak.outcome_filled.values[np.where(mask)]

#np.sum(all_leak.outcome_leak.isnull())

In [39]:
pickle.dump(all_leak, open('cvleak8-10fold.pkl', 'wb'))